In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
reviews = pd.read_csv('Reviews.csv')

In [3]:
print (reviews)


              Timestamp  How would you score the class   \
0    2/15/2019 13:42:01                               1   
1    2/15/2019 14:37:29                               1   
2    2/15/2019 11:06:42                               2   
3    2/15/2019 11:08:46                               2   
4    2/15/2019 11:06:05                               3   
5    2/15/2019 11:06:52                               3   
6    2/15/2019 11:07:21                               3   
7    2/15/2019 11:07:34                               3   
8    2/15/2019 11:09:17                               3   
9    2/15/2019 11:10:14                               3   
10   2/15/2019 11:56:54                               3   
11   2/15/2019 13:07:27                               3   
12   2/15/2019 13:40:02                               3   
13   2/15/2019 13:40:37                               3   
14   2/15/2019 14:38:11                               3   
15   2/15/2019 15:30:21                               3 

In [4]:
reviews.head()

,Timestamp,How would you score the class,Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?),"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [5]:
reviews.shape


(133, 8)

In [6]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 8 columns):
Timestamp                                                                                                                          133 non-null object
How would you score the class                                                                                                      133 non-null int64
Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)    132 non-null object
Polarity (-1, 0, 1)                                                                                                                0 non-null float64
Suggestions (0 or 1)                                                                                                               0 non-null float64
Suggestions                                                                                                                        0 non-null fl

In [7]:
reviews.describe()


,How would you score the class,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
count,133.000000,0.0,0.0,0.0,0.0,0.0
mean,4.255639,NaN,NaN,NaN,NaN,NaN
std,0.813281,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN
25%,4.000000,NaN,NaN,NaN,NaN,NaN
50%,4.000000,NaN,NaN,NaN,NaN,NaN
75%,5.000000,NaN,NaN,NaN,NaN,NaN
max,5.000000,NaN,NaN,NaN,NaN,NaN


In [8]:
reviews.rename(
    columns={
        'Explain why you gave that score? (What is good about the class? What is bad about the class? What could make the class better?)' : 'rev', 'How would you score the class ' : 'stars'
    },
    inplace=True
)
reviews.head()

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN


In [9]:
reviews['rev'] = reviews['rev'].apply(str)
## This converts the column values of reviews to strings

In [10]:
reviews['text length'] = reviews['rev'].apply(len)
reviews.head()

#this creates a new column that will apply the text length of review

,Timestamp,stars,rev,"Polarity (-1, 0, 1)",Suggestions (0 or 1),Suggestions,Flag (0 1),Flag Data,text length
0,2/15/2019 13:42:01,1,we suck. ...,NaN,NaN,NaN,NaN,NaN,167
1,2/15/2019 14:37:29,1,no naptime / cheese ohnononononnononononononon...,NaN,NaN,NaN,NaN,NaN,86
2,2/15/2019 11:06:42,2,Because of mr liberman and i need to calm down...,NaN,NaN,NaN,NaN,NaN,117
3,2/15/2019 11:08:46,2,I gave this class a 2 because it is too fast f...,NaN,NaN,NaN,NaN,NaN,268
4,2/15/2019 11:06:05,3,Mr. Liberman is a fun teacher but we have to r...,NaN,NaN,NaN,NaN,NaN,113


In [11]:
X = reviews['rev']
y = reviews['stars']

In [36]:
print(type(X))

<class 'pandas.core.series.Series'>


In [12]:
# tokenizer = Tokenizer(num_words=2000)
# tokenizer.fit_on_texts(X)
# sequences = tokenizer.texts_to_sequences(X)
# one_hot_results = tokenizer.texts_to_matrix(X, mode='binary')
# word_index = tokenizer.word_index

# print('Found %s unique tokens.' % len(word_index))
# print(tokenizer)
# print(one_hot_results)
# print(word_index)


In [13]:
import re 
def getwordlist(text):
    # Convert words to lower case and split them
    text = text.lower().split()    
    text = " ".join(text)    
    #Remove Special Characters
    text=re.sub(r'[^a-z\d ]','',text)    
    #Replace Numbers
    text=re.sub(r'\d+','n',text)
    # Return a list of words
    return(text)

comments = []
for text in X:
    comments.append(getwordlist(text))

In [14]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(comments) 
sequences = tokenizer.texts_to_sequences(comments)


word_index = tokenizer.word_index
print('unique words %s' % len(word_index))

train_data = pad_sequences(sequences, maxlen=2500, padding='post')
print('Train data shape:', train_data.shape)



unique words 648
Train data shape: (133, 2500)


In [44]:
test_comments = []
for t in text:
    test_comments.append(getwordlist(t))
    
test_sequences = tokenizer.texts_to_sequences(test_comments)
test_data = pad_sequences(test_sequences, maxlen=2500, padding='post')
print('Test data shape:', test_data.shape)

Test data shape: (56, 2500)


In [15]:
#Glove Vectors loaded

embeddings_index = {}

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

 

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [16]:
#do not need to do pipelining

#want to use the model.fit- parse the train data and then specify the epochs and batch size.


# prepare embeddings for layer
from keras.layers.embeddings import Embedding

words_max = len(word_index)+1

embedding_matrix = np.zeros((len(word_index)+1, 100))

for word, i in word_index.items():

    if i >= words_max:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

In [17]:
##this is where add Karuna Code
# Create an non re-trainable embedding layer with embedding size 100

nb_words = len(word_index)+1

embedding_layer = Embedding(nb_words,100,

        weights=[embedding_matrix],

        input_length=2500,

        trainable=True)

In [22]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    model.summary()

    return model

## Sets up the model for how much data is processed at once.

In [23]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2500, 100)         64900     
_________________________________________________________________
lstm_3 (LSTM)                (None, 2500, 300)         481200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 639,605
Trainable params: 639,605
Non-trainable params: 0
_________________________________________________________________


In [24]:
#x is the comment, or reviews, can be many columns,  y is the labels

In [25]:
hist = model.fit(train_data, y, 
        epochs=2, batch_size=128)

Epoch 1/2
133/133 [==============================] - 121s 913ms/step - loss: 18.7669
Epoch 2/2
133/133 [==============================] - 132s 992ms/step - loss: 18.7669


In [38]:
#testdata = X.loc[:10]
lables = y[:10]

In [32]:
print(lables)
print(testdata)

0     1
1     1
2     2
3     2
4     3
5     3
6     3
7     3
8     3
9     3
10    3
Name: stars, dtype: int64
0     we suck.                                      ...
1     no naptime / cheese ohnononononnononononononon...
2     Because of mr liberman and i need to calm down...
3     I gave this class a 2 because it is too fast f...
4     Mr. Liberman is a fun teacher but we have to r...
5     I like this class because my friend is in this...
6     It's fun and everything but we watch too many ...
7     It's kind of boring and has not  been any  pro...
8     TO MU WRITINGGGG---------\n\n\n\n\n-----pppppp...
9     i gave this score because this class is helpfu...
10    I have no opinion, the class was fine! To make...
Name: rev, dtype: object


In [41]:
loss = model.evaluate(test_data, lables, verbose=1)
#print('Accuracy: %f' % (accuracy*100))

10/10 [==============================] - 5s 490ms/step


In [42]:
text = "this is a test to see if it works, Karuna is the best!!!"

In [45]:
prediction = model.predict(test_data)

In [46]:
prediction


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)